In [5]:
# Python code for Monte Carlo sampling of EQ scenarios
## Written by Mohammad Amin Nabian, mnabia2@illinois.edu, June 2016

## Import libraries
import numpy as np
from numpy import linalg as LA
import random
import gpxpy.geo
from scipy import stats
import warnings

# Inputs:
variable = 'Magnitude'     # Magnitude, Epicenter, Depth
num_scenario = 10000
M0 = 6.8
Mmax = 7.5
lat0 = 37.12  #37.04
lon0 = -121.9 #-121.88
beta = 0.76
if variable == 'Magnitude':
    lower, upper, scale = M0, Mmax, 1./beta
    Magnitude = stats.truncexpon(b=(upper-lower)/scale, loc=lower, scale=scale)
    #Magnitude = stats.uniform(loc=lower, scale=upper-lower)
    M = Magnitude.rvs(num_scenario)
    lat = lat0 * np.ones(num_scenario)
    lon = lon0 * np.ones(num_scenario)
if variable == 'Epicenter':   # needs to be completed later
    mean = [0, 0]
    cov = [[1, 0], [0, 1]]
    lat, lon = np.random.multivariate_normal(mean, cov, num_scenario).T
    M = M0 * np.ones(num_scenario)
    
# Step 1: Get the bridge location (longitude and latitude), class (HWB1 through HWB28), number of spans (N), 
# skew angle (alpha), span width (W), bridge length (L), and maximum span length (Lmax).
sid = [32,232,231,231,226,227,225,74,654,392,214,221,264,701,267,655,213,211,209, #18
       205,74,654,236,82,83,31,97,688,507,34,43,456,48,14,693,139,154,464,130]
nbridges = len(sid)
strg='HAZUS_DATA.txt'
x = np.loadtxt(strg, delimiter='\t' , skiprows=1 , usecols=[8,9,10,11,12,24,25])
W = x[sid,0]
N = x[sid,1]
L = x[sid,2]
Lmax = x[sid,3]
alpha = x[sid,4]
location = x[sid,5:7]
ID = ['CA011153','CA011545','CA011544','CA011544','CA011535','CA011536','CA011532','CA011227','CA012216','CA011871',
     'CA011488','CA011511','CA011609','CA012266','CA011614','CA012217','CA011488','CA011486','CA011484','CA011477',
     'CA011227','CA012216','CA01554','CA011243','CA011245','CA011152','CA011270','CA012252','CA012042','CA011155',
     'CA011174','CA011979','CA011182','CA011134','CA012258','CA011349','CA011373','CA011993','CA011327']
Class = ['10','3','22','22','22','3','3','23','10','23','10','22','11','10','4','10','22','10','10','4',
         '23','10','3','10','4','10','18','3','19','10','10','11','3','3','28','10','10','28','10']
# HAZUS tables
# Table 7.3
A = np.zeros(nbridges); B = np.zeros(nbridges)
for i in range (nbridges):
    if Class[i] in ['1','2','3','4','5','6','7','14','17','18','19']: A[i] = 0.25; B[i] = 1.
    if Class[i] in ['8','10','20','22']: A[i] = 0.33; B[i] = 0.
    if Class[i] in ['9','11','16','21','23']: A[i] = 0.33; B[i] = 1.
    if Class[i] in ['12','13']: A[i] = 0.09; B[i] = 1.
    if Class[i] in ['15']: A[i] = 0.05; B[i] = 0.
    if Class[i] in ['24','25']: A[i] = 0.20; B[i] = 1.
    if Class[i] in ['26','27','13']: A[i] = 0.10; B[i] = 0.
    if Class[i] in ['28']: A[i] = 0.10; B[i] = 0.
    warnings.warn("K3D coeeficients not available for one or more of your bridges of class HWB28. For now values for HWB27 are being used.")
# Table 7.7
median = np.zeros(nbridges)
for i in range (nbridges):
    if Class[i] in ['1']: median[i] = 0.70
    if Class[i] in ['2','7','10','11','14','16','19','22','23']: median[i] = 1.10
    if Class[i] in ['3','4','28']: median[i] = 1.20
    if Class[i] in ['5','12','17','24']: median[i] = 0.45
    if Class[i] in ['6','13','18','25']: median[i] = 0.60
    if Class[i] in ['8','20']: median[i] = 0.55
    if Class[i] in ['9','21']: median[i] = 1.30
    if Class[i] in ['15','26','27']: median[i] = 0.75

# Step 2: Evaluate the soil-amplified shaking at the bridge site. That is, get the peak ground
# acceleration (PGA), spectral accelerations (Sa[0.3 sec] and Sa[1.0 sec]).
# Calculate the PGA for each scenario for each bridge
# GMPE function
def GMPEGK15(M,R,VS30,F,Q_0,Bdepth,flag):     # GK15
    if flag == 'm':
        amp = 1.
    else:
        amp = 1./1.12
    ## GPME for PGA (GK15PGA)
    c1 = 0.14
    c2 = -6.25
    c3 = 0.37
    c4 = 2.237
    c5 = -7.542
    c6 = -0.125
    c7 = 1.19
    c8 = -6.15
    c9 = 0.6
    c10 = 0.345
    bv = -0.24
    VA = 484.5
    # lnPGA = G1 + G2 + G3 + G4 + G5
    G1 = np.log((c1 * np.arctan(M+c2) + c3) * F)
    Ro = c4 * M + c5
    Do = c6 * np.cos(c7 * (M + c8)) + c9
    G2 = -0.5 * np.log((1-R/Ro)**2. + 4. * (Do**2.) * (R/Ro))
    G3 = -c10 * R / Q_0
    G4 = bv * np.log(VS30/VA)
    A_Bdepth = 1.077/np.sqrt((1.-(1.5/(Bdepth+0.1))**2.)**2.+4.*0.7**2.*(1.5/(Bdepth+0.1))**2.)
    A_bdist = 1/np.sqrt((1.-(40./(R+0.1))**2.)**2.+4.*0.7**2.*(40/(R+0.1))**2)
    G5 = np.log(1 + A_Bdepth * A_bdist)
    lnPGA = G1 + G2 + G3 + G4 + G5
    PGA = np.exp(lnPGA) * amp
    ## GMPE for Spectral Shape (GK15SS)
    t = [0.01,0.02,0.022,0.025,0.029,0.03,0.032,0.035,0.036,0.04,0.042,0.044,0.045,0.046,0.048,0.05,
        0.055,0.06,0.065,0.067,0.07,0.075,0.08,0.085,0.09,0.095,0.1,0.11,0.12,0.13,0.133,0.14,0.15,
        0.16,0.17,0.18,0.19,0.2,0.22,0.24,0.25,0.26,0.28,0.29,0.3,0.32,0.34,0.35,0.36,0.38,0.4,
        0.42,0.44,0.45,0.46,0.48,0.5,0.55,0.6,0.65,0.667,0.7,0.75,0.8,0.85,0.9,0.95,1.,
        1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.,2.2,2.4,2.5,2.6,2.8,3.,3.2,3.4,3.5,
        3.6,3.8,4,4.2,4.4,4.6,4.8,5.]
    m1 = -0.0012
    m2 = -0.38
    m3 = 0.0006
    m4 = 3.9
    a1 = 0.01686
    a2 = 1.2695
    a3 = 0.0001
    Dsp = 0.75
    t1 = 0.001
    t2 = 0.59
    t3 = -0.0005
    t4 = -2.3
    s1 = 0.001
    s2 = 0.077
    s3 = 0.3251
    if M < 5.5:
        M = 5.5
    F1 = np.zeros(len(t))
    F2 = np.zeros(len(t))
    Y = np.zeros(len(t))
    for j in range (len(t)):
        I = (a1*M+a2) * np.exp(a3*R)
        mu = m1*R + m2*M + m3*VS30 + m4
        S = s1*R - (s2*M+s3)
        Tsp_o = max(0.3,np.abs(t1*R + t2*M + t3*VS30 + t4))
        zay = 1.763 - 0.25 * np.arctan(1.4*(Bdepth-1.))
        F1[j] = I * np.exp(-0.5*((np.log(t[j])+mu)/S)**2.)
        F2[j] = 1./np.sqrt((1-(t[j]/Tsp_o)**zay)**2. + 4.*Dsp**2.*(t[j]/Tsp_o)**zay)
        Y[j] = F1[j] + F2[j]
    Sigma = np.zeros(len(t))
    for i in range (len(t)):
        Sigma[i] = max(0.668 + 0.0047 * np.log(t[i]) , 0.8 + 0.13 * np.log(t[i]))
    ## SA = GK15SS * exp(GK15PGA)
    SA = np.zeros(len(t)+1)
    SA[0] = PGA
    SA[1:len(t)+1] = Y[0:len(t)] * np.exp(lnPGA) * amp
    t = np.insert(t,0,0)
    Sigma = np.insert(Sigma,0,0.66)
    return t, SA, Sigma

VS30_table = [239.,216.5,216.5,222.,222.,222.,229.,189.,189.,189.,234.,234.,234.,234.,234.,234.,234.,234.,234.,234.,
     200.,203.,203.,216.,216.,261.,271.,299.,263.,263.,231.,231.,231.,300.1,300.1,299.,299.,299.,446.9]
s = (nbridges,num_scenario) ; PGA = np.zeros(s) ; Sa3 = np.zeros(s) ; Sa10 = np.zeros(s)
for j in range (num_scenario):
    for i in range (nbridges):
        R = gpxpy.geo.haversine_distance(lat[j], lon[j], location [i,0], location [i,1]); R = R/1000.
        t, SA, Sigma = GMPEGK15(M[j],R,VS30_table[i],1.28,150.,0.,'m')
        
        PGA_value = SA[0]
        Sa3_value = SA[45]
        Sa10_value = SA[68]
        c3 = 1; c10 = 1.
        if Sa3_value < 0.25:
            c3 = 1.6
        elif Sa3_value < 0.50:
            c3 = 1.4
        elif Sa3_value < 0.75:
            c3 = 1.2
        elif Sa3_value < 1.0:
            c3 = 1.1
        else:
            c3 = 1.0
        if Sa10_value < 0.1:
            c10 = 2.4
        elif Sa10_value < 0.2:
            c10 = 2.0
        elif Sa10_value < 0.3:
            c10 = 1.8
        elif Sa10_value < 0.4:
            c10 = 1.6
        else:
            c10 = 1.5
            
#         PGA[i,j] = np.exp(np.random.normal(np.log(PGA_value), 0.669**2.)) #0.669
#         Sa3[i,j] = np.exp(np.random.normal(np.log(c3*Sa3_value), 0.694**2.)) #0.694
#         Sa10[i,j] = np.exp(np.random.normal(np.log(c10*Sa10_value), 0.807**2.)) #0.807
        Sa3[i,j] = c3*SA[45]
        Sa10[i,j] = c10*SA[68]

# Step 3: Evaluate the modification factors
if any (alpha > 90.):
    warnings.warn("Skew angle greater than 90. For now the absolute values are being used.")
K_skew = np.sqrt(np.sin(np.abs(90.*np.ones(nbridges)-alpha)*np.pi/180.))
K3D = 1. + A/(N-B)
for i in range (nbridges):
    if K3D[i] == np.Inf: K3D[i] = 1. ########!!!!!!!!!!WARNING!!!!!!!!!!########

# Step 4: Modify the ground shaking medians for the “standard” fragility curves in Table 7.7:
new_median = median*K_skew*K3D

# Step 5: Use the new medians along with the dispersion  = 0.6 to evaluate the ground shaking- related
# damage state probabilities.
g = 9.807
s = (nbridges,num_scenario); P = np.zeros(s)   #survival probability, equal to 1-p(at least extensive)
for i in range (nbridges):
    for j in range (num_scenario):
        P[i,j] = 1. - stats.lognorm.cdf(Sa10[i,j]*g, s=0.6, loc=0., scale=new_median[i]*g)  #arguments: (x=amplified Sa[1.0sec],s=0.6(beta), loc=0., scale=new median))
np.savetxt('SURVIVALS_Evaluate.txt', P)

/Users/Imate/ProgramFiles/Anaconda3/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:64: UserWarning: K3D coeeficients not available for one or more of your bridges of class HWB28. For now values for HWB27 are being used.
/Users/Imate/ProgramFiles/Anaconda3/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:201: RuntimeWarning: divide by zero encountered in true_divide
